In [1]:
#dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#import api key
from config import weather_api_key

#incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#output file (csv)
output_data_file = "output_data/cities.csv"

#range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)
lat_range = (-90, -18)
lng_range = (-180, -150)

#formatting for bold text
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

In [2]:
#printing out the minneapolis json to see the data
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}appid={weather_api_key}&q="
response = requests.get(query_url + "minneapolis").json()
response

{'coord': {'lon': -93.26, 'lat': 44.98},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 261.95,
  'feels_like': 257.57,
  'temp_min': 259.26,
  'temp_max': 264.82,
  'pressure': 1040,
  'humidity': 78},
 'visibility': 16093,
 'wind': {'speed': 1.5, 'deg': 180},
 'clouds': {'all': 1},
 'dt': 1579541284,
 'sys': {'type': 1,
  'id': 4984,
  'country': 'US',
  'sunrise': 1579527876,
  'sunset': 1579561375},
 'timezone': -21600,
 'id': 5037649,
 'name': 'Minneapolis',
 'cod': 200}

In [3]:
#List for holding lat_lngs and cities
lat_lngs = []
cities = []

#create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=-18.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=-150.000, size=1500)
lat_lngs = zip(lats, lngs)

# Add a one second interval between queries to stay within API query limits
time.sleep(1)

#identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

#print the city count to confirm sufficient count
len(cities)

11

In [7]:
#perform a weather check on each city using a series of successive API calls.
#include a print log of each city as it's being processed (with the city number and city name).

#base url
url = "http://api.openweathermap.org/data/2.5/weather?"
#build partial query url
query_url = f"{url}appid={weather_api_key}&q="

city_names = []
city_numbers = []
city_info = []
city_lats = []
cloudiness = []
country = []
date = []
humidity = []
city_lngs = []
max_temp = []
wind_speed = []
i = 1
set_number = 1



print("Beginning Data Retrieval") 
print("--------------------------------")



for j, city in enumerate(cities):
    
    if (j % 50 == 0 and j >= 50):
        set_number = set_number + 1
        i = 0
    print(f"Processing Record {i} of Set {set_number} | {city}")
    i=i+1
   
    response = requests.get(query_url + city).json()
    
    # Add a one second interval between queries to stay within API query limits
    time.sleep(1)
    
    try:
        city_names.append(city)
        city_numbers.append(response['id'])
        city_lats.append(response['coord']['lat'])
        cloudiness.append(response['clouds']['all'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        city_lngs.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])
        #city_info.append({"Lat": city_lats})
        
        #city_lat = city_weather["coord"]["lat"]
        #city_lng = city_weather["coord"]["lon"]
        #city_max_temp = city_weather["main"]["temp_max"]
        #city_humidity = city_weather["main"]["humidity"]
        #city_clouds = city_weather["clouds"]["all"]
        #city_wind = city_weather["wind"]["speed"]
        #city_country = city_weather["sys"]["country"]
        #city_date = city_weather["dt"]
        
        
        #city_info.append({"City": city, 
                          #"Lat": city_lat, 
                          #"Lng": city_lng, 
                          #"Max Temp": city_max_temp,
                          #"Humidity": city_humidity,
                          #"Cloudiness": city_clouds,
                          #"Wind Speed": city_wind,
                          #"Country": city_country,
                          #"Date": city_date})
        
        
    
    except:
        print("City " + color.BOLD + "not " + color.END + "found. Skipping...")
        pass

print("------------------------")
print("Data Retrieval Complete")
print("------------------------")
print(city_info)

Beginning Data Retrieval
--------------------------------
Processing Record 1 of Set 1 | vaini
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | avarua
Processing Record 4 of Set 1 | alofi
Processing Record 5 of Set 1 | pangai
Processing Record 6 of Set 1 | neiafu
Processing Record 7 of Set 1 | tevaitoa
Processing Record 8 of Set 1 | avera
Processing Record 9 of Set 1 | papara
Processing Record 10 of Set 1 | moerai
Processing Record 11 of Set 1 | vaitape
------------------------
Data Retrieval Complete
------------------------
[]


In [9]:
weather_dict = {"City": city_names,
                "Cloudiness": cloudiness,
                "Country": country,
                "Date": date,
                "Humidity": humidity,
                "Lat": city_lats,
                "Lng": city_lngs,
                "Max Temp": max_temp,
                "Wind Speed": wind_speed}
weather_data = pd.DataFrame(weather_dict)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,vaini,20,TO,1579541582,78,-21.20,-175.20,297.15,4.60
1,mataura,94,NZ,1579541583,95,-46.19,168.86,286.48,0.89
2,avarua,100,CK,1579541584,88,-21.21,-159.78,300.15,5.10
3,alofi,75,NU,1579541585,83,-19.06,-169.92,298.15,3.10
4,pangai,20,TO,1579541586,73,-19.80,-174.35,298.15,3.10
5,neiafu,20,TO,1579541588,100,-18.65,-173.98,293.15,3.79
6,tevaitoa,100,PF,1579541589,76,-16.78,-151.50,300.36,5.35
7,avera,1,US,1579541590,38,33.19,-82.53,280.37,3.60
8,papara,40,BR,1579541493,58,-3.98,-38.73,304.15,5.10
9,moerai,100,PF,1579541592,80,-22.43,-151.33,299.26,7.32


In [6]:
#export city data into a csv file
#make into a dataframe


city_df = pd.DataFrame(city_info)


city_df.to_csv(output_data_file, index_label="City_ID")



FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'